In [21]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [35]:
mri = pd.read_csv('unrestricted_hcp_freesurfer.csv', index_col='Subject')
behav = pd.read_csv('hcp_behavioral_data.csv', index_col='Subject')
joined = mri.join(behav, how='inner', lsuffix='_mri', rsuffix='_behav')

In [40]:
# Setup X and y
y = joined['Gender_mri']
X = joined.drop(['Gender_mri', 'Gender_behav'], axis=1)

# Choose only numeric columns
X = X.select_dtypes(include=['number'])

In [50]:
# Run rbf kernel svm on the data

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# print only string columns
print(X_test.select_dtypes(include=['object']).columns)
# train['NEORAW_01']

Index([], dtype='object')


In [52]:
# Scale the data

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Fit the model
clf = SVC(kernel='rbf')
clf.fit(X_train, y_train)
# Calculate training accuracy
y_pred = clf.predict(X_train)
print("TRAIN:")
print(accuracy_score(y_train, y_pred))

# Predict the test set
y_pred = clf.predict(X_test)
print("TEST:")
print(accuracy_score(y_test, y_pred))